# `20180516a` - JSON Server - version un0_e

In [1]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import time

matplotlib.use('Agg')

BigJS0N = {}
BigJS0N["firmware_md5"]="fa6a7560ade6d6b1149b6e78e0de051f"
BigJS0N["firmware_version"]="e_un0"
BigJS0N["data"]=[]
BigJS0N["registers"]={}
BigJS0N["parameters"]={}

def CreateDACCurve(Deb,Fin,CurveType):
    n = 200/5
    DACValues = []
    for k in range(n+1):
        if CurveType:
            val = int(Deb+1.0*k*(Fin-Deb)/n)
        else:
            val = int((Fin-Deb)*k**3/n**3+Deb)
        DACValues.append(val) 
    DACValues[-1] = 0
    DACValues[-2] = 0
    return DACValues,len(DACValues)
    
    
def SetDACCurve(spi,DACValues):
    if len(DACValues) < 43: # to correct
        for i in range(len(DACValues)):
            if (DACValues[i] >= 0) and (DACValues[i] < 1020):
                WriteFPGA(spi,16+i,DACValues[i]/4)
            else:
                 WriteFPGA(spi,16+i,0)
            #print 16+i,len(DACValues)
    return 0

def WriteFPGA(spi,adress,value):
    spi.xfer([0xAA] )
    spi.xfer([adress] )
    spi.xfer([value] )
    BigJS0N["registers"][int(adress)]=value
    
def StartUp():
    GPIO.setmode(GPIO.BCM)
    PRESET = 23 ## Reset for the FPGA
    IO4 = 26 ## 26 is the output connected to 
    GPIO.setup(PRESET,GPIO.OUT)
    GPIO.setup(IO4,GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
    print "Reset GPIO 23 - Low 1s"
    GPIO.output(PRESET,GPIO.LOW)

    time.sleep(3)
    print "Reset GPIO 23 - High 0.2s"
    GPIO.output(PRESET,GPIO.HIGH)
    time.sleep(0.2)

    CS_FLASH = 7 ## Reset for the FPGA
    GPIO.setup(CS_FLASH,GPIO.OUT)
    print "CS_FLASH on "+str(CS_FLASH)+" going low."
    GPIO.output(CS_FLASH,GPIO.LOW)
    
    
    spi = spidev.SpiDev()
    spi.open(0,0) # CS2 - FPGA, on CE1 = IO4
    spi.mode = 0b01
    print "spi.cshigh is " + str(spi.cshigh)
    print "spi mode is " + str(spi.mode)
    spi.max_speed_hz = 2000000
    print "spi maxspeed is "+str(spi.max_speed_hz)+"hz"
    return spi

def TestSPI(spi,ncycles):
    i = 0
    while i < ncycles:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        time.sleep(0.5)
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
        time.sleep(0.5)  
        i = i+1

def LoopSPI(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode

def LoopAcq(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x00) # Doing 1 shot 
        WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
        WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software
        time.sleep(0.001) # sleep 1ms
    
def ClearMem(spi):
    WriteFPGA(spi,0xEF,0x01) # To access memory

def ConfigSPI(spi):
    # Setup FPGA values by default
    setPon(200,spi)              # Set PulseOn
    setPulsesDelay(100,spi)      # Set Lengh between Pon and Poff: 100ns
    setPoff(2000,spi)            # Setting Poff 2us
    #setDACConstant(20,spi)   # gain at 20mV (2%)
    WriteFPGA(spi,0xEC,0x33) # 33 acquisitions
    setDeltaAcq(7000,spi)    # 7us
    #WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
    WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
    WriteFPGA(spi,0xED,0x03) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
    WriteFPGA(spi,0xEE,0xA0) # How many cycles in countinious mode
    print "Config FPGA done!"

def setDACConstant(mV,spi):
    if mV > 1000:
        mV = 1000
    elif mV < 0:
        mV = 0   
    hmV = mV/4
    print "Gain:", mV," mV -- ",hex(hmV)
    WriteFPGA(spi,0xEC,hmV) # Voltage gain control: 0V to 1V
    
def setPon(POn,spi):
    if POn > 2500:
        POn = 2500
    elif POn < 0:
        POn = 0
    HPon = POn* 128 / 1000
    BigJS0N["parameters"]["Pon"] = int(POn)
    print "Pulse width:", POn," ns -- ",hex(HPon)
    WriteFPGA(spi,0xE0,HPon) # set sEEPon
    
def setPulsesDelay(DeltaPP,spi):
# Set Lengh between Pon and Poff
    if DeltaPP > 2500:
        DeltaPP = 2500
    elif DeltaPP < 0:
        DeltaPP = 0
    HPP =DeltaPP * 128 / 1000
    #print  hex(HPP)
    BigJS0N["parameters"]["PulsesDelay"] = int(DeltaPP)
    print "Pulses delay:", DeltaPP," ns -- ",hex(HPP)
    WriteFPGA(spi,0xD0,HPP) # set sEEPon

def setPoff(sEEPoff,spi):
    # Sets the damping length.
    POff = sEEPoff * 128 / 1000
    #print sEEPoff,POff
    POffMSB, POffLSB = 0x00FF&POff/256,0x00FF&POff 
    print "Poff:", sEEPoff," ns -- ",hex(POffMSB),hex(POffLSB)
    BigJS0N["parameters"]["Poff"] = int(sEEPoff)
    WriteFPGA(spi,0xE1,POffMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE2,POffLSB) # set sEEPon LSB

    # Setting Poff to Acq delay sEEDelayACQ
def setDeltaAcq(DeltaAcq,spi):
    if DeltaAcq > 255*255:
        DeltaAcq = 254*254
    elif DeltaAcq < 0:
        DeltaAcq = 0
    hDA = DeltaAcq * 128 / 1000
    hDAMSB, hDALSB = hDA/255 , 0x00FF&hDA 
    print "Delay between:",DeltaAcq,"ns -- ", hex(hDAMSB),hex(hDALSB)
    BigJS0N["parameters"]["DeltaAcq"] = int(DeltaAcq)
    WriteFPGA(spi,0xE3,hDAMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE4,hDALSB) # set sEEPon LSB
    
def SetLengthAcq(LAcqI,spi):
    LAcq = LAcqI * 128 / 1000
    #print LAcq,hex(LAcq),hex(LAcqI)
    BigJS0N["parameters"]["LengthAcq"] = int(LAcqI)
    LAcqMSB, LAcqLSB = 0x00FF&LAcq/256 , 0x00FF&LAcq
    print "Acquisition length: ", LAcq, " ns -- ",hex(LAcqMSB),hex(LAcqLSB)
    WriteFPGA(spi,0xE5,LAcqMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE6,LAcqLSB) # set sEEPon LSB

def setPeriodAcq(lEPeriod,spi):
    lEPNs = lEPeriod*128/1000 #ns
    EPNsMSB, EPNs, EPNsLSB = 0x00FF&lEPNs/(256*256),0x00FF&lEPNs/256,0x0000FF&lEPNs 
    print "Period between two acquisitions:", lEPeriod,"us --", hex(EPNsMSB),hex(EPNs),hex(EPNsLSB) 
    BigJS0N["parameters"]["PeriodAcq"] = int(lEPeriod)
    WriteFPGA(spi,0xE7,EPNsMSB) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,EPNs) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,EPNsLSB) # Period of one cycle LSB

def setPulseTrain(Pon,Pdelay,Poff,DelayAcq,Acq,spi):
    setPon(Pon,spi)
    setPulsesDelay(Pdelay+Pon,spi)
    setPoff(Poff+Pdelay+Pon,spi)
    setDeltaAcq(DelayAcq+Poff+Pdelay+Pon,spi)
    SetLengthAcq(Acq+DelayAcq+Poff+Pdelay+Pon,spi)
    #SetLengthAcq(Acq,spi)

/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:1314: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
spi = StartUp()
print("------")
ConfigSPI(spi)
print("------")
t1 = 200
t2 = 95
t3 = 2000
t4 = 7000
t5 = 130000
setPulseTrain(t1,t2,t3,t4,t5,spi)
N = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.


Reset GPIO 23 - Low 1s
Reset GPIO 23 - High 0.2s
CS_FLASH on 7 going low.
spi.cshigh is False
spi mode is 1
spi maxspeed is 2000000hz
------
Pulse width: 200  ns --  0x19
Pulses delay: 100  ns --  0xc
Poff: 2000  ns --  0x1 0x0
Delay between: 7000 ns --  0x3 0x80
Config FPGA done!
------
Pulse width: 200  ns --  0x19
Pulses delay: 295  ns --  0x25
Poff: 2295  ns --  0x1 0x25
Delay between: 9295 ns --  0x4 0xa5
Acquisition length:  17829  ns --  0x45 0xa5


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.


In [3]:
#LoopSPI(spi)

In [4]:
TestSPI(spi,3) # LED2 clignote 3x

In [5]:
BigJS0N

{'data': [],
 'firmware_md5': 'fa6a7560ade6d6b1149b6e78e0de051f',
 'firmware_version': 'e_un0',
 'parameters': {'DeltaAcq': 9295,
  'LengthAcq': 139295,
  'Poff': 2295,
  'Pon': 200,
  'PulsesDelay': 295},
 'registers': {208: 37,
  224: 25,
  225: 1,
  226: 37,
  227: 4,
  228: 165,
  229: 69,
  230: 165,
  235: 0,
  236: 51,
  237: 3,
  238: 160}}

In [6]:
Curve = CreateDACCurve(0,1000,True)[0] # Beginning, Ending, Linear (if False, expo)
print Curve,len(Curve)
#setDACConstant(0x77,spi)
SetDACCurve(spi,Curve)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500, 525, 550, 575, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 900, 925, 950, 0, 0] 41


0

In [23]:
N=0

In [27]:
BigJS0N["N"] = N
BigJS0N["V"]="25"
f = 0x00
t1 = 200
t2 = 100
t3 = 2000
t4 = 300-t1-t2+10
t5 = 200000

Curve = CreateDACCurve(200,500,True)[0]
SetDACCurve(spi,Curve)

BigJS0N["experiment"]={}
BigJS0N["experiment"]["id"] = "20180516a"
BigJS0N["experiment"]["description"]="Classical experiment with calibration piezo"

setPulseTrain(t1,t2,t3,t4,t5,spi)

WriteFPGA(spi,0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)

WriteFPGA(spi,0xEC,0x01) # Doing 10 lines
WriteFPGA(spi,0xEB,0x01) # Doing one line if 0, several if 1


if BigJS0N["registers"][235]: # means it's set to 1, ie that's multiples lines
    NLines = BigJS0N["registers"][236]
else:
    NLines = 1

print "Lines number: "+str(NLines)
    
WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

Fech = int(64/((1+f)))
LAcq = t5/1000 #ns to us
Nacq = LAcq * Fech * NLines

print "F="+ str(Fech)+ "-> "+str(Nacq) + ' samples',Nacq,LAcq,Fech,NLines 

A = []
for i in range(2*Nacq+1):
    A.append ( spi.xfer([0x00] )[0] )

a = np.asarray(A).astype(int)

BigJS0N["data"] = A
BigJS0N["experiment"]["target"] = "calibration rig"
BigJS0N["experiment"]["position"] = "0"

N = N+1
print N
with open(BigJS0N["experiment"]["id"]+'-'+str(N)+".json", 'w') as outfile:
    json.dump(BigJS0N, outfile)
        
print "Done"

Pulse width: 200  ns --  0x19
Pulses delay: 300  ns --  0x26
Poff: 2300  ns --  0x1 0x26
Delay between: 2310 ns --  0x1 0x27
Acquisition length:  25895  ns --  0x65 0x27
Lines number: 1
F=64-> 12800 samples 12800 200 64 1
4
Done
